In [1]:
import numpy as np
# Autoreload
%load_ext autoreload
%autoreload 2

# Imports
import matplotlib.pyplot as plt
import pandas as pd

from ecodynelec.parameter import Parameter  # Import the class to manipulate parameters
from etude_stat.analysis_functions import generate_data
from etude_stat.analysis_functions import plot_metrics, format_data_0

# Basic configuration
my_config = Parameter()
my_config.freq = "M"
my_config.ctry = ['AT', 'CH', 'DE', 'FR', 'IT']
#my_config.ctry = ['ES', 'CH', 'DE', 'FR', 'BE', 'GB', 'IT']
my_config.target = 'CH'
my_config.data_cleaning = True
my_config.path.generation = "./test_data/downloads/generations/"
my_config.path.exchanges = "./test_data/downloads/exchanges/"
my_config.residual_local = False


In [2]:
from ecodynelec.pipelines import execute

year = '2021'
my_config.start = year + '-01-01 00:00'
my_config.end = year + '-12-31 23:59'
my_config.freq = 'M'
my_config.data_cleaning = True
my_config.net_exchanges = False
my_config.residual_global = False
my_config.residual_local = False
mat = execute(my_config, is_verbose=True)

Load auxiliary datasets...
Load generation data...
	Generation data.
Data loading: 13.01 sec.
Memory usage table: 196.13 MB
Autocomplete...               5/5)...
	Completed.                              
Extraction raw generation: 7.29 sec.             
	Extraction time: 20.68 sec.
{'AT':                      Biomass_AT  Fossil_Brown_coal/Lignite_AT   
2021-01-01 00:00:00       180.0                           NaN  \
2021-01-01 00:15:00       180.0                           NaN   
2021-01-01 00:30:00       180.0                           NaN   
2021-01-01 00:45:00       180.0                           NaN   
2021-01-01 01:00:00       180.0                           NaN   
...                         ...                           ...   
2021-12-31 22:45:00       140.0                           NaN   
2021-12-31 23:00:00       336.0                           NaN   
2021-12-31 23:15:00       336.0                           NaN   
2021-12-31 23:30:00       336.0                           Na

In [11]:
mat

{'CH':               Mix_AT    Mix_CH    Mix_DE    Mix_FR    Mix_IT Mix_Other   
 2021-01-01  0.074349  1.001397  0.203127  0.064884  0.004413  0.025276  \
 2021-02-01  0.067337  1.000963  0.150023  0.142912  0.003885  0.020141   
 2021-03-01  0.055954  1.000513  0.172701  0.114363  0.000028  0.022127   
 2021-04-01  0.031745  1.001477  0.180523  0.059014  0.008972  0.016643   
 2021-05-01  0.029057  1.001641  0.089547   0.10033  0.010514  0.007635   
 2021-06-01  0.011779  1.001043  0.078981  0.066829   0.00177  0.004063   
 2021-07-01  0.008793  1.000905  0.077277  0.074458       0.0  0.005316   
 2021-08-01  0.054993  1.001089  0.078082  0.081654  0.000037  0.008807   
 2021-09-01   0.02882  1.001023  0.199327  0.107807  0.000523  0.014921   
 2021-10-01  0.012332  1.000475  0.276326  0.170292  0.000056  0.018042   
 2021-11-01  0.033852  1.001124  0.297872  0.078557  0.007905   0.02699   
 2021-12-01  0.052854  1.001377   0.23608  0.075172  0.013996  0.025372   
 
            Bioma

In [4]:
#### 1.1 Download and generate data files
# This can done once, then the electrical mix and impact files will be saved and can be loaded directly (see 1.2)
years = ['2021'] #['2018', '2019', '2020', '2021']
raws, mixs, impacts = generate_data(my_config, years)

#### 1.2 Load data from saved files
#If the files for the years of interest have already been generated, they can be loaded directly in this section
# todo update impacts, mixs = load_data(my_config, years)

#### 1.3 Pre-processing the data
# Here we reformat the data to make it easier to manipulate and plot
raw_production = format_data_0(raws)
electricity_mix = format_data_0(mixs)
electricity_impact = format_data_0(impacts)

Compute  2021
Load auxiliary datasets...
Load generation data...
	Generation data.
Data loading: 15.21 sec.
Memory usage table: 196.13 MB
Autocomplete...               5/5)...
	Completed.                              
Extraction raw generation: 7.49 sec.             
	Extraction time: 23.08 sec.
	4/4 - Resample exchanges to MS steps...
Get and reduce importation data...
	Cross-border flow data.
Data loading: 6.52 sec..
Memory usage table: 79.02 MB
Autocomplete...               ...
	Completed.                              
Extraction raw import: 1.53 sec.             
	Extraction time: 8.18 sec.
imported {'AT':                      AL  AM  AT  AZ  BA  BE  BG  BY   CH           CZ  ...   
2021-01-01 00:00:00 NaN NaN NaN NaN NaN NaN NaN NaN  0.0  1336.400024  ...  \
2021-01-01 00:15:00 NaN NaN NaN NaN NaN NaN NaN NaN  0.0  1300.699951  ...   
2021-01-01 00:30:00 NaN NaN NaN NaN NaN NaN NaN NaN  0.0  1265.000000  ...   
2021-01-01 00:45:00 NaN NaN NaN NaN NaN NaN NaN NaN  0.0  1229.300049 

KeyError: 'Climate Change'

In [ ]:
df = electricity_mix['raw_df'].resample('Y').mean()
local_cols = df.mean().nlargest(20).index.tolist()
for f in local_cols:
    df[f] = df[f].map(lambda x: f'{x * 100:.0f} %')
df.index = pd.Series(years)

yearly = raw_production['raw_df'].resample('Y').sum().sum(axis=1).map(lambda x: f'{x / 1000:.2f} GWh')
yearly.index = pd.Series(years)
print(yearly)

In [ ]:
yearly = raw_production['raw_df']
yearly = yearly.loc[[d.year == 2021 for d in yearly.index]]
yearly = yearly.resample('M').sum().sum(axis=1).map(lambda x: f'{x / 1000:.2f} GWh')
yearly.index = [f'{d.month_name()}' for d in yearly.index]
print(yearly)

In [72]:

from ecodynelec.pipelines import get_prod_mix_impacts

year = '2021'
my_config.start = year + '-01-01 00:00'
my_config.end = year + '-12-31 23:59'
my_config.data_cleaning = True
my_config.net_exchanges = False
my_config.residual_global = False
my_config.residual_local = True
raw_prodExch, m, i = get_prod_mix_impacts(my_config, is_verbose=True)

Load auxiliary datasets...
Load generation data...
	Generation data.
Data loading: 12.81 sec.
Memory usage table: 196.13 MB
Autocomplete...               5/5)...
	Completed.                              
Extraction raw generation: 7.13 sec.             
	Extraction time: 20.33 sec.
	4/4 - Resample exchanges to MS steps...
Get and reduce importation data...
	Cross-border flow data.
Data loading: 6.24 sec..
Memory usage table: 79.02 MB
Autocomplete...               ...
	Completed.                              
Extraction raw import: 1.56 sec.             
	Extraction time: 7.91 sec.
Resample exchanges to MS steps...
                Mix_AT_CH  Mix_CH_CH     Mix_DE_CH    Mix_FR_CH   Mix_IT_CH   
2021-01-01  610334.125000        0.0  1.586971e+06   528655.750   36077.125  \
2021-02-01  512517.156250        0.0  1.062904e+06  1074109.875   29993.000   
2021-03-01  438019.468750        0.0  1.267982e+06   887801.000     211.000   
2021-04-01  203405.781250        0.0  1.152229e+06   383550.75

In [62]:
raw_prodExch

,Residual_Hydro_AT,Residual_Other_AT,Biomass_AT,Fossil_Brown_coal/Lignite_AT,Fossil_Coal-derived_gas_AT,Fossil_Gas_AT,Fossil_Hard_coal_AT,Fossil_Oil_AT,Fossil_Oil_shale_AT,Fossil_Peat_AT,...,Solar_IT,Waste_IT,Wind_Offshore_IT,Wind_Onshore_IT,Mix_AT_IT,Mix_CH_IT,Mix_DE_IT,Mix_FR_IT,Mix_IT_IT,Mix_Other_IT
2019-01-01,0.0,0.0,228243.0,0.0,0.0,1.573819e+06,176104.093750,0.0,0.0,0.0,...,857244.0,30604.0,0.0,2354898.0,104377.000000,1777838.000,0.0,897269.0,0.0,462319.718750
2019-02-01,0.0,0.0,207732.0,0.0,0.0,1.244398e+06,183729.000000,0.0,0.0,0.0,...,1315594.0,28024.0,0.0,2334119.0,106972.296875,1995518.625,0.0,1477383.0,0.0,555725.687500
2019-03-01,0.0,0.0,229459.0,0.0,0.0,6.657015e+05,125398.296875,0.0,0.0,0.0,...,1876446.0,33155.0,0.0,2431774.0,81614.000000,1986757.000,0.0,1473763.0,0.0,617499.812500
2019-04-01,0.0,0.0,210922.0,0.0,0.0,5.275099e+05,97397.796875,0.0,0.0,0.0,...,1745102.0,27394.0,0.0,1462302.0,117574.000000,1247986.000,0.0,1007081.0,0.0,537432.125000
2019-05-01,0.0,0.0,201955.0,0.0,0.0,7.867550e+04,74575.898438,0.0,0.0,0.0,...,1829966.0,19581.0,0.0,1640922.0,109815.000000,1329383.000,0.0,1458433.0,0.0,550231.187500
2019-06-01,0.0,0.0,171830.0,0.0,0.0,1.799184e+05,76191.101562,0.0,0.0,0.0,...,2345592.0,19646.0,0.0,979915.0,110946.000000,1491997.000,0.0,1442165.0,0.0,502753.593750
2019-07-01,0.0,0.0,175275.0,0.0,0.0,4.915083e+05,124530.796875,0.0,0.0,0.0,...,2330406.0,29919.0,0.0,1236981.0,112632.000000,1864205.000,0.0,1621813.0,0.0,375959.875000
2019-08-01,0.0,0.0,153995.0,0.0,0.0,7.041834e+05,7631.899902,0.0,0.0,0.0,...,2323640.0,31787.0,0.0,715482.0,77600.000000,1493279.000,0.0,1027250.0,0.0,92238.968750
2019-09-01,0.0,0.0,162604.0,0.0,0.0,9.275528e+05,26454.000000,0.0,0.0,0.0,...,1812461.0,32804.0,0.0,1158772.0,92832.000000,1783877.000,0.0,1241121.0,0.0,180311.421875
2019-10-01,0.0,0.0,151196.0,0.0,0.0,1.234911e+06,133638.000000,0.0,0.0,0.0,...,1369617.0,28623.0,0.0,1024188.0,114980.000000,2456780.000,0.0,1232803.0,0.0,325538.906250


In [63]:
raw_prod_dict = {}
for target in my_config.ctry:
    target_dict = {col: raw_prodExch[col] for col in raw_prodExch.columns if col.endswith(target)}
    raw_prod_dict[target] = pd.DataFrame.from_dict(target_dict)

In [64]:
r = raw_prod_dict
r

{'AT':             Residual_Hydro_AT  Residual_Other_AT  Biomass_AT   
 2019-01-01                0.0                0.0    228243.0  \
 2019-02-01                0.0                0.0    207732.0   
 2019-03-01                0.0                0.0    229459.0   
 2019-04-01                0.0                0.0    210922.0   
 2019-05-01                0.0                0.0    201955.0   
 2019-06-01                0.0                0.0    171830.0   
 2019-07-01                0.0                0.0    175275.0   
 2019-08-01                0.0                0.0    153995.0   
 2019-09-01                0.0                0.0    162604.0   
 2019-10-01                0.0                0.0    151196.0   
 2019-11-01                0.0                0.0    141022.0   
 2019-12-01                0.0                0.0    150172.0   
 
             Fossil_Brown_coal/Lignite_AT  Fossil_Coal-derived_gas_AT   
 2019-01-01                           0.0                         0.0  \
 

In [65]:
print(r['CH']['Mix_Other_CH'])

2019-01-01    0.0
2019-02-01    0.0
2019-03-01    0.0
2019-04-01    0.0
2019-05-01    0.0
2019-06-01    0.0
2019-07-01    0.0
2019-08-01    0.0
2019-09-01    0.0
2019-10-01    0.0
2019-11-01    0.0
2019-12-01    0.0
Name: Mix_Other_CH, dtype: float32


In [66]:
p = r['CH'].sum().sum()

In [67]:
s = 0
for t in my_config.ctry:
    s = s + r[t]['Mix_CH_'+t].sum()

In [68]:
s

30109232.375

In [69]:
p - s

66036888.515625

In [70]:
export = 0
impo = 0
for t in my_config.ctry:
    if t != 'CH':
        impo = impo + r['CH']['Mix_'+t+'_CH'].sum()
        export = export + r[t]['Mix_CH_'+t].sum()
local = r['CH'].drop(r['CH'].loc[:,[c for c in r['CH'].columns if c.startswith('Mix')]].columns, axis=1).sum().sum()

In [71]:
(local, impo, export, local + impo - export, local+ impo)

(69351356.7890625,
 26794764.1015625,
 30109232.375,
 66036888.515625,
 96146120.890625)